# CHAPTER 1: The Python Data Model

# Introduction

1. Python có tính nhất quán cao (consistency)

2. Sự khác biệt trong thiết kế cú pháp

- Ví dụ: `len(collection)` thay vì `collection.len()` ban đầu có thể thấy lạ.
- Đây là dấu hiệu cho thấy **Python ưu tiên cú pháp rõ ràng, đồng nhất**, và điều này đến từ **Data Model**.


3. Python Data Model là gì?
- Là **bộ quy tắc/hợp đồng** (API) để bạn khiến object của mình **tương tác trơn tru với cú pháp và tính năng của Python**
- Nó giống như một **framework** mà bạn viết code theo những **phương thức đặc biệt** (`__dunder__ methods`) để tích hợp vào ngôn ngữ.


4. Special Methods (Magic / Dunder Methods)
- Tên bắt đầu và kết thúc bằng dấu `__` (ví dụ: `__getitem__`, `__str__`, `__len__`...).
- Python sẽ **gọi các phương thức này khi bạn sử dụng các cú pháp quen thuộc**, ví dụ:

  * `obj[key]` ⟶ gọi `obj.__getitem__(key)`
  * `len(obj)` ⟶ gọi `obj.__len__()`


5. Các tính năng bạn có thể hỗ trợ thông qua Data Model

Bằng cách định nghĩa các dunder method, có thể khiến object của mình hỗ trợ:

| Tính năng                        | Dunder method tiêu biểu                     |
| -------------------------------- | ------------------------------------------- |
| Iteration (duyệt)                | `__iter__`, `__next__`                      |
| Collections (chỉ mục, độ dài...) | `__getitem__`, `__len__`, `__contains__`    |
| Attribute access (thuộc tính)    | `__getattr__`, `__setattr__`, `__delattr__` |
| Operator overloading (toán tử)   | `__add__`, `__eq__`, `__lt__`, etc.         |
| Function & method call           | `__call__`                                  |
| Object creation & destruction    | `__new__`, `__init__`, `__del__`            |
| String representation            | `__str__`, `__repr__`                       |
| Context manager (`with`)         | `__enter__`, `__exit__`                     |


6. “Dunder” là gì?
- “Dunder” = **double underscore**, ví dụ `__getitem__` = **dunder-getitem**
- Đây là cách nói tắt phổ biến và thân thiện trong cộng đồng Python.


Mini-review

> **Câu hỏi:** Khi bạn viết `len(my_obj)`, điều gì thực sự xảy ra trong nền?

> **Trả lời:** Python gọi `my_obj.__len__()` nếu phương thức đó được định nghĩa!


# A Pythonic Card Deck

Một ví dụ đơn giản nhưng biểu hiện tác dụng to lớn của method `__getitem__` và `__len__`.

Một đối tượng chứa 52 lá bài:

In [7]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [
            Card(rank, suit)
            for suit in self.suits
            for rank in self.ranks
        ]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]


beer_card = Card('7', 'diamonds')
print(beer_card)


# Nếu định nghĩa __len__ trong class thì có thể sử dụng len() để lấy số lượng phần tử
deck = FrenchDeck()
len(deck)


# Nếu định nghĩa __getitem__ trong class thì có thể sử dụng [] để lấy phần tử
print(deck[0])  # Lấy thẻ đầu tiên
print(deck[-1])  # Lấy thẻ cuối cùng


# Nếu muốn lấy ngẫu nhiên, Python đã có random.choice
import random
print(random.choice(deck))  # Lấy một thẻ ngẫu nhiên từ bộ bài

Card(rank='7', suit='diamonds')
Card(rank='2', suit='spades')
Card(rank='A', suit='hearts')
Card(rank='K', suit='spades')


# How Special Methods Are Used

🧠 **Cách dùng đúng các phương thức đặc biệt (special methods)**

* **Không nên gọi trực tiếp** như `my_object.__len__()` — thay vào đó, dùng hàm dựng sẵn như `len(my_object)`.
* Python gọi `__len__`, `__iter__`, `__str__`, v.v. **ngầm định** thông qua các câu lệnh như `for x in y`, `str(x)`, `len(x)`.
* Với kiểu dữ liệu built-in, **CPython tối ưu hóa tốc độ** bằng cách bỏ qua lời gọi hàm và đọc trực tiếp từ cấu trúc C bên dưới.
* Trừ khi làm **metaprogramming**, bạn nên **viết** các special method hơn là gọi chúng.
* `__init__` là ngoại lệ thường được gọi trực tiếp khi dùng `super()`.
* Tránh tạo **thuộc tính kiểu `__foo__` tùy ý**, vì chúng có thể được dùng bởi Python trong tương lai.

## Emulating Numeric Types

➕ **Mô phỏng kiểu số học với special methods**

* Bạn có thể dùng **các phương thức đặc biệt** như `__add__`, `__mul__`, `__abs__`, `__repr__` để hỗ trợ **toán tử số học** trong lớp do người dùng định nghĩa.

* Ví dụ: lớp `Vector` 2D dùng để:

  * **Cộng vector** (`+`) → `v1 + v2` → `Vector(4, 5)`
  * **Nhân với số** (`*`) → `v * 3` → `Vector(9, 12)`
  * **Tính độ dài** (`abs(v)`) → dùng `__abs__`, như `abs(Vector(3, 4)) = 5.0`

* Dù `complex` có thể dùng làm vector 2D, lớp `Vector` này **có thể mở rộng lên nhiều chiều (n-dimensional)** → sẽ tiếp tục ở Chương 14.

In [ ]:
from math import hypot

class Vector:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return f'Vector({self.x}, {self.y})'
    
    def __abs__(self):
        return hypot(self.x, self.y)
    
    def __add__(self, other):
        return bool(abs(self))
    
    def __add__(self, other):
        x = self.x + other.x
        y = self.y + other.y
        return Vector(x, y)
    
    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)
    

X = Vector(3, 4)
Y = Vector(1, 2)

# Kiểm tra các special methods
print(X)  # In ra Vector(3, 4)
print(abs(X))  # In ra độ dài của vector X
print(X + Y)  # In ra Vector(4, 6)
# Nhân vector với một số
print(X * 2)  # In ra Vector(6, 8)


Vector(3, 4)
5.0
Vector(4, 6)
Vector(6, 8)


📌 **Special methods thường được gọi ngầm định**

* Dù lớp `Vector` cài đặt các phương thức như `__add__`, `__abs__`, `__repr__`, **chúng không được gọi trực tiếp** trong mã.
* Python **tự động gọi** chúng khi ta dùng các hàm hoặc toán tử như `+`, `abs()`, `print()`,…
* Phần tiếp theo sẽ **phân tích chi tiết từng phương thức** đã được dùng trong ví dụ.

## String Representation

🧾 `__repr__` vs `__str__`: Đại diện chuỗi của đối tượng

* **`__repr__`**:

  * Gọi bởi `repr()` → dùng trong **console, debugger, `%r`, hoặc `!r`** trong định dạng chuỗi.
  * Mục tiêu: **rõ ràng, không mơ hồ**, có thể dùng để **tái tạo đối tượng** nếu được.
  * Ví dụ: `Vector(3, 4)` → nên hiển thị đúng `Vector(3, 4)`.

* **`__str__`**:

  * Gọi bởi `str()` hoặc `print()` → dành cho **người dùng cuối**, dễ đọc hơn.

* ✅ Nếu chỉ viết một hàm, hãy ưu tiên `__repr__`. Python sẽ dùng nó thay cho `__str__` nếu `__str__` không có.

* Ngoài ra, khi định dạng chuỗi trong `__repr__`, dùng `%r` là thói quen tốt → hiển thị rõ kiểu giá trị (vd: `1` vs `'1'`).

## Arithmetic Operators

✖️➕ **Toán tử `+` và `*` trong lớp `Vector`**

* Lớp `Vector` cài đặt `__add__` và `__mul__` để hỗ trợ:

  * **Cộng vector**: `v1 + v2`
  * **Nhân vô hướng**: `v * 3`

* Các phép toán này **không thay đổi đối tượng gốc**, mà tạo **đối tượng mới** → đúng với quy tắc của toán tử trung tố (infix operators).

* **Lưu ý**:

  * `Vector * number` hoạt động
  * Nhưng `number * Vector` **không hoạt động** vì chưa có `__rmul__`.

✅ Sẽ bổ sung `__rmul__` để xử lý **nhân theo hướng ngược** (commutative).


## Boolean Value of a Custom Type

✅ **Kiểm tra tính đúng sai (truth value) với `__bool__`**

* Python dùng `bool(x)` để xác định một đối tượng có "truthy" (đúng) hay "falsy" (sai).

* Mặc định, **object tự định nghĩa luôn là `True`**, trừ khi bạn cài:

  * `__bool__()` → ưu tiên gọi trước
  * `__len__()` → gọi nếu không có `__bool__`, và trả về `False` nếu độ dài = 0

* Với `Vector`, `__bool__` được định nghĩa là `False` nếu độ dài (magnitude) bằng 0.

* **Tối ưu hiệu suất**:

  ```python
  def __bool__(self):
      return bool(self.x or self.y)
  ```

  → Nhanh hơn vì tránh tính căn bậc hai, nhưng **khó đọc hơn**.

## Overview of Special Methods

🧩 **Toàn cảnh về Special Methods trong Python**

* Python có **83 special methods** (theo Python Language Reference).

  * **47 trong số đó** dùng cho **toán tử** (số học, so sánh, bitwise,...)

#### 🗂 **Bảng tổng quan (không bao gồm toán tử):**

* **Biểu diễn chuỗi**: `__repr__`, `__str__`, `__format__`, `__bytes__`
* **Chuyển kiểu số**: `__abs__`, `__bool__`, `__int__`, `__float__`, `__hash__`,...
* **Mô phỏng collection**: `__len__`, `__getitem__`, `__setitem__`, `__contains__`
* **Lặp**: `__iter__`, `__next__`, `__reversed__`
* **Callable**: `__call__`
* **Context manager**: `__enter__`, `__exit__`
* **Quản lý đối tượng**: `__init__`, `__new__`, `__del__`
* **Quản lý thuộc tính**: `__getattr__`, `__setattr__`, `__get__`,...

#### ➕ **Các nhóm toán tử:**

* **Số học**: `__add__`, `__sub__`, `__mul__`,...
* **So sánh**: `__eq__`, `__lt__`, `__ge__`,...
* **Đảo toán tử** (reversed): `__radd__`, `__rmul__`,...
* **Toán tử gán cộng dồn**: `__iadd__`, `__imul__`,...
* **Bitwise**: `__and__`, `__or__`, `__xor__`,...

✅ Các toán tử đảo (`__r*__`) và toán tử gán (`__i*__`) sẽ được giải thích kỹ hơn ở **Chương 13**.


## Why len Is Not a Method

📏 **Vì sao `len(x)` không phải là `x.len()`?**

* Lý do chính: **tính thực tiễn quan trọng hơn sự thuần nhất** (Zen of Python).
* Với các kiểu built-in (như `list`, `str`), `len(x)` **rất nhanh** vì chỉ đọc trực tiếp một trường trong struct C — **không gọi method**.
* Nhờ có `__len__`, bạn vẫn có thể làm `len()` hoạt động với **object tự định nghĩa** → đảm bảo tính **nhất quán của ngôn ngữ**.

📌 So sánh thêm:

* `len()` và `abs()` hoạt động như **toán tử một ngôi** (unary operator).
* Trong ngôn ngữ ABC (tiền thân của Python), dùng `#s` thay cho `len(s)`.

# Chapter Summary

📚 **Tổng kết Chương 1 – Python Data Model**

* **Special methods** giúp **object của bạn hành xử như kiểu built-in** → viết mã theo phong cách **Pythonic**.

* Hai special method quan trọng đầu tiên:

  * `__repr__`: dùng cho **debug**
  * `__str__`: dùng để **hiển thị cho người dùng**

* **Mô phỏng sequence** là một trong những ứng dụng phổ biến nhất của special methods → sẽ học sâu hơn ở:

  * **Chương 2**: tận dụng các kiểu sequence có sẵn
  * **Chương 10**: tự xây dựng sequence (Vector nhiều chiều)

* Python hỗ trợ **nhiều toán tử số học**, kể cả **toán tử đảo chiều (`__r*__`) và gán tăng (`__i*__`)** → sẽ học ở **Chương 13**.

* Các special method khác sẽ được giới thiệu dần **xuyên suốt cả sách**.

# Further Reading

📚 **Tài liệu đọc thêm về Python Data Model**

🔖 **Nguồn chính thống**

* **“Data Model”** trong *Python Language Reference* là nguồn gốc chính.
* Một số tài liệu kinh điển:

  * *Python in a Nutshell* (Alex Martelli) – mô tả cơ chế truy cập thuộc tính rất chi tiết.
  * *Python Essential Reference* & *Python Cookbook* (David Beazley) – tập trung vào Python 3.

🧠 **Metaobject Protocol & Magic Methods**

* Python gọi là **special methods**, Ruby gọi là **magic methods** — nhưng thật ra **không hề “ma thuật”**: người dùng có thể định nghĩa giống như core dev.
* JavaScript có các hành vi **“magic thật”**, ví dụ: thuộc tính chỉ đọc không thể tạo thủ công (trước ES5.1).

🧬 **Metaobject Protocol là gì?**

* Là “API cho cấu trúc lõi của ngôn ngữ”.
* Python và Ruby có **metaobject protocol mạnh mẽ** → cho phép mở rộng theo phong cách lập trình mới (như aspect-oriented).
* Tác giả AMOP, **Gregor Kiczales**, cũng là cha đẻ của **AspectJ** (mở rộng Java theo hướng aspect-oriented).

💬 **Soapbox nhỏ**

* “Data model” và “Object model” thực ra là cùng một thứ.

  * Python dùng từ “data model” trong tài liệu chính thức nên sách giữ theo cách gọi đó.